In [2]:
############################ 
# 1.  Social Network 
##########################

library(igraph)
library(RColorBrewer)
library(tidyverse)

ERROR: Error in library(igraph): there is no package called ‘igraph’


In [ ]:
# We first read in the Enron dataset (stringsAsFactors=F means we do not treat strings as factors)
# The nodes data frame contains information about all people in Enron, each treated as a node in our network
nodes <- read.csv("enron_nodes.csv", stringsAsFactors=F)
str(nodes)
# The pairs data frame includes all the pairs of employees who were in frequent communication
# The 'ct' column incicates the number of emails sent to each other over a pre-specified period
pairs <- read.csv('emails.csv')
str(pairs)
# As you can see, all the pairs in the dataset have sent at least 10 emails to each other
min(pairs$ct)

# We now create an undirected graph, where the vertices/nodes come from the 'nodes' variable and the edges come from the 'pairs' dataset
enron.G <- graph.data.frame(pairs, directed=FALSE, vertices=nodes)  

# Let us now plot the graph
# When we plot the graph, there is some randomness regarding where each node will be located
# Thereforem we will fix the seed
set.seed(144)
plot(enron.G, vertex.label=NA, vertex.size=3, vertex.color = "black")
# From the plot, we see some ``outliers'', i.e., those nodes that do not have any correspondences with other nodes.

# We can also add some color to the graph.
# For instance, let us color by position in the company
color.plot <- rep('white',nrow(nodes)) #we create a vector of values 'white'
# We now specify a color for each position in the company
color.plot[nodes$Type=="Trading"] <- 'blue'
color.plot[nodes$Type=="Legal"] <- 'green'
color.plot[nodes$Type=="Executive"] <- 'red'
color.plot[nodes$Type=="Pipeline"] <- 'brown'
color.plot[nodes$Type=="Risk Management"] <- 'cyan'
color.plot[nodes$Type=="Regulatory Affairs"] <- 'black'
color.plot[nodes$Type=="Logistics"] <- 'purple'
color.plot[nodes$Type=="Accounting"] <- 'magenta'

# Let us apply our colors:
set.seed(144)
plot(enron.G, vertex.label=NA, vertex.size=3, vertex.color = color.plot)

## We first look at the degrees of all the nodes
# This is done with the degree() function
enron.degree <- degree(enron.G)
# Let us look at the output:
enron.degree
# It is a vector with one value for each employee, which indicates the number of connections of that employee

set.seed(144)
# We set the node size proportional to its degree for better visualization
plot(enron.G, vertex.label=NA, vertex.size=0.3*enron.degree,vertex.color = color.plot)

# We display the top three people with the highest degree
# The order() function takes the order of indices such that the underlying vector is sorted
# The decreasing=TRUE argument specifies that we are sorting the vector in decreasing order
nodes[order(enron.degree,decreasing=TRUE)[1:3],]

## We then look at the closeness of all the nodes
# A challenge here is that closeness is ill defined for disconnected graphs
# We therefore decompose the graph into complete sub-graphs
# components(enron.G)$csize denotes the size of each subgraph, and which.max() finds the index of the largest one
# and components(enron.G)$membership indicates the membership of each node
nodes.in.subgraph <- components(enron.G)$membership == which.max(components(enron.G)$csize) # extract the nodes in the largest subgraph
subgraph <- induced_subgraph(enron.G, nodes.in.subgraph)  # extract the subgraph from enron.G 

# We can calculate the closeness as follows:
closeness(subgraph)
# But note that we are only getting values for the nodes in the main subgraph:
length(closeness(subgraph))
# So we need to fix a value of 0 for the other ones
# We will do that in a 'cl' variable
cl <- rep(0, nrow(nodes))           # all the nodes in enron.G intially has closeness 0
# only compute the closeness of the nodes in the subgraph; the rest nodes have closeness 0, which makes sense:
# if a node is disconnected from the main subgraph, then it takes infinite number of steps to reach other nodes
# so its closeness is 0
cl[nodes.in.subgraph] <- closeness(subgraph)
set.seed(144)
plot(enron.G, vertex.label=NA, vertex.size=2000*cl, vertex.color = color.plot)

# We display the top three people with the highest closeness centrality
nodes[order(cl,decreasing = T)[1:3],]

## We now proceed as for "degree" for betweenness and pagerank

## Betweenness
enron.betweenness <- betweenness(enron.G)
set.seed(144)
plot(enron.G, vertex.label=NA, vertex.size=0.02*enron.betweenness,vertex.color = color.plot)
nodes[order(enron.betweenness,decreasing = T)[1:3],]

## PageRank
page.rank.score = page.rank(enron.G)$vector
set.seed(144)
# Note that with the page.rank() function, we have multiple outputs, so we need to take the one called 'vector'
plot(enron.G, vertex.label=NA, vertex.size=500*page.rank.score,vertex.color = color.plot)
nodes[order(page.rank.score,decreasing = T)[1:3],]

##### Community detection

# igraph makes this pretty easy too!
# There are a few ways to cluster communities, and different algorithms
# It is a hard problem, so some methods are fast and less accurate, some are more.

# We'll use the cluster_spinglass function in this lecture
# Due to how the algorithm works, cluster_spinglass only works if we have a single connected component
# Therefore, we will now be using our subgraph with 141 nodes rather than the full graph with 148 nodes

set.seed(144)
spinglass = cluster_spinglass(subgraph, spins = 100)
# Note: spins is an integer constant, an upper limit for the number of communities.

# Now we have a community for each node (if you set a
# different random seed you may have different communities)
community = spinglass$membership
# This gives you the community to which each node belongs
community

# We can look at the size of each community as follows
table(community)
# We obtain 10 communities in total

# We can look at the breakdown by position
table(nodes$Type[nodes.in.subgraph], community)

# And we can also look at the modularity of our community detection:
modularity <- spinglass$modularity # modularity = p.within - p.random

# Finally, we will plot the subgraph
# We will include a different color for each community
#Remember: We have 10 communities:
max(community)
# So we will define 10 colors
color.communities <- brewer.pal(max(community),"Spectral")
# The 'brewer.pal' function defines a palette of the pre-specified number of colors

# Let's plot!
set.seed(144)
plot(subgraph, vertex.label=NA, vertex.size=5, vertex.color=color.communities[community])

## Now, we include "ground truth" communities, based on job positions

set.seed(144)
plot(subgraph, vertex.label=NA, vertex.size=ifelse(V(subgraph)$Type == "Trading", 8, 3),# if the employee does trading, the node has size 10; and 3 otherwise 
     vertex.color=color.communities[community])

set.seed(144)
plot(subgraph, vertex.label=NA, vertex.size=ifelse(V(subgraph)$Type == "Trading" & V(subgraph)$Region=="W", 8, 3),# if the employee does trading and in west region, the node has size 10; and 3 otherwise 
     vertex.color=color.communities[community])

set.seed(144)
plot(subgraph, vertex.label=NA, vertex.size=ifelse(V(subgraph)$Type == "Executive", 8, 3),# if the employee does executive jobs, the node has size 10; and 3 otherwise 
     vertex.color=color.communities[community])

set.seed(144)
plot(subgraph, vertex.label=NA, vertex.size=ifelse(V(subgraph)$Type == "Pipeline", 10, 3),# if the employee does pipeline jobs, the node has size 10; and 3 otherwise 
     vertex.color=color.communities[community])

set.seed(144)
plot(subgraph, vertex.label=NA, vertex.size=ifelse(V(subgraph)$Type == "Accounting", 10, 3),# if the employee does accounting, the node has size 10; and 3 otherwise 
     vertex.color=color.communities[community])

set.seed(144)
plot(subgraph, vertex.label=NA, vertex.size=ifelse(V(subgraph)$Type == "Legal", 10, 3),# if the employee does legal jobs, the node has size 10; and 3 otherwise 
     vertex.color=color.communities[community])

############################################################
# 2. Collaborative filtering for building recommender systems.
############################################################

library(softImpute) # collaborative filtering package

# The following command reads the MovieLens data.
ratings <- read.csv('ratings.csv')

max(ratings$movieID)

# Let's investigate the dataset...
head(ratings)
# The first column lists the user IDs
# The second column lists the movie IDs
# The third column lists the ratings
# The next columns provides the time of the rating

library(plyr)
count(ratings, "userID")

# Let us split the dataset into a training set and a test set, as usual
# The overwhelming majority (99%) of the dataset will go into the training set.
# This is because the data are already very sparse, so we want to use as much of it as possible
set.seed(144)
train.idx <- sample(seq_len(nrow(ratings)), 0.99*nrow(ratings))
train.ratings <- ratings[train.idx,]
test.ratings <- ratings[-train.idx,]

# Let us transform the training set into a matrix.
# The rows correspond to users
# The columns correspond to movies
# The entries correspond to ratings
# The issue is that ratings are missing for a lot of user-movie pairs (this is precisely what we want to estimate!)
# This is accomplished using the "Incomplete" function
mat.train <- Incomplete(train.ratings[, 1], train.ratings[, 2], train.ratings[, 3]) #(user, movie, rating)
dim(train.ratings)
# Let us see... Each dot repressents a missing entry. There are a lot of them!
mat.train

# We are now ready to apply our collaborative filtering algorithm.
# This is done with the softImpute function, as follows:
set.seed(144)
fit <- softImpute(mat.train, rank.max=9, lambda=0, maxit=1000)
# The "rank.max" argument corresponds to the maximal number of archetypes (in lecture, we called it "k")
# Let's not worry about lambda for now (it has to do with the Lasso procedure---stay tuned!)
# The "maxit" argument specifies a maximum number of iterations for the fitting procedure

# There are three primary outputs to the softImpute function
# First, the "u" element is a matrix of size "number of users" x "number of archetypes"
# It represents the linear combination of archetypes for each user
fit$u   # n x k (#user x #archetype)
# The other two elements are called "v" and "d"
# "v" is of size "number of movies" * "number of archetypes"; it indicates the relative preference of each archetype for each movie
# "d" is of size "number of archetypes"; it is just a scaling factor (diagonal matrix)
# Their product is of size "number of movies" * "number of archetypes"
# It indicates the rating of each archetype for each movie
fit$v * fit$d  # m x k  (#movie x #archetype)

# We are now ready to make predictions!
# For that, we need a list of users (test.ratings[, 1]) and a list of movies (test.ratings[, 2])
# We call the function "impute"
pred.insample.0 <- impute(fit, train.ratings[, 1], train.ratings[, 2])
pred.outsample.0 <- impute(fit, test.ratings[, 1], test.ratings[, 2])

# You can verify manually that the predicted value is just the sumproduct of the weights (fit$u) and the archetype ratings (fit$v * fit*d)
# sum(fit$u[test.ratings[1,1],] * fit$v[test.ratings[1,2],] * fit$d)
# pred.outsample.0[1]

# Let us show the histogram of these values
hist(pred.insample.0)
hist(pred.outsample.0)

# The issue is that some values are lower than 1 and some are higher than 5.
# We simply treat all values lower than 1 as 1's and all values higher than 5 as 5's.
# This is done as follows:
pred.insample <- pmax(pmin(pred.insample.0, 5), 1)
pred.outsample <- pmax(pmin(pred.outsample.0, 5), 1)

# We now compute the in-sample and out-of-sample performance of the model:
R2.insample <- 1 - sum((pred.insample-train.ratings$rating)^2)/sum((mean(train.ratings$rating) - train.ratings$rating)^2)
R2.outsample <- 1 - sum((pred.outsample-test.ratings$rating)^2)/sum((mean(train.ratings$rating) - test.ratings$rating)^2)
R2.insample
R2.outsample

############################################################
# 3. Ensemble modeling for enhancing recommender systems. (Optional)
############################################################

# As discussed in class, the prediction from the collaborative filtering model can be integrated into a regression model
# In other words, the regression model will leverage the outputs of the collaborative filtering model, as well as additional features on the users and the movies
# By doing so, we combine two predictive methods together. This is called ensemble modeling.

# Let us first import additional data

### USER DATA
user.info <- read.csv("users.csv")

# Let's investigate the dataset...
head(user.info)
str(user.info)
# The first column lists the user IDs
# The second column lists the age
# The third column lists the job category
# The fourth column lists the zipcode
# The last column lists the gender (1 for male, 0 for female)

### ZIPCODE-INCOME DATA
zip.med <- read.csv("MedianZIP-3.csv")

# Let's investigate the dataset...
head(zip.med)
str(zip.med)
# The first column lists the zipcodes
# The second column lists the median income
# The third column lists the mean income
# The fourth column lists the population

# A bit of data cleaning...
# Remove commas in numbers and convert into numbers
zip.med$Median <- as.numeric(gsub(",", "", zip.med$Median))
# Make sure all zip codes have 5 digits
zip.med$Zip <- str_pad(zip.med$Zip, 5, "left", "0")
# Match income from zip.med with the zipcodes given in user.info
# The challenge is that, due to missing data entries, we will have "NA" values
# For these, we input the average median income, taken over all zipcodes for which we have values
user.info$MedianIncome <- NA                # new field
m <- match(user.info$ZipCode, zip.med$Zip)
user.info$MedianIncome[!is.na(m)] <- zip.med$Median[m[!is.na(m)]]
user.info$MedianIncome[is.na(user.info$MedianIncome)] <- mean(user.info$MedianIncome, na.rm=TRUE) # ave median income

# In the model presented in class, we also included additional user-level features
# by mapping zipcode into an urban/rural indicator and into a region (Pacific, Eastern, etc.)
# Here, we simplify the model by ignoring these additional data (which were not the main drivers of the predictions anyway)

### MOVIE GENRE DATA
genre.mat <- read.csv('genre.csv')

# Let us investigate the dataset
head(genre.mat)
# Each row lists a movie (the row ids correspond to the movie ids in the ratings.csv file)
# Each column lists a genre
# a value of 1 means that the movie is classified into the genre

# Let us combine all this information into a model matrix
# The function model.matrix() works similarly to the regression functions, except that it does not fit any model---it just creates the matrix
# Here, we consider all the variables, except zipcode itself (we used it to get acces to the median income data, but zipcode itself is not informative)
MM <- as.data.frame(model.matrix(~.-ZipCode, data=user.info)[,-(1:2)])  # exclude the first two cols

# Let us investigate this matrix
head(MM)

### Re-construction of a training set and a test set with all the relevant variables, including:
# the user info data
# the movie genre data
# the rating data,
# the previous predictions from the collaborative filtering ('CF') model, and
# some additional data fields, such as weekday, month, year, and hour of day
full.mat <- cbind(MM[ratings$userID,], genre.mat[ratings$movieID,])  # see ratings
full.train <- full.mat[train.idx,]   # previous training indices
full.train$CF <- pred.insample       # new field
full.train <- cbind(train.ratings[,c("rating", "wday", "mon", "year", "hour")],full.train) # prepend date info
names(full.train) <- make.names(names(full.train)) # ensure names to be syntatically correct

full.test <- full.mat[-train.idx,]
full.test$CF <- pred.outsample
full.test <- cbind(test.ratings[,c("rating", "wday", "mon", "year", "hour")],full.test)
names(full.test) <- make.names(names(full.test))

# We now run our linear regression model, using all the aforementioned predictors:
mod.lm.full <- lm(rating~., data=full.train)
summary(mod.lm.full)
R2.insample

# Our previous in-sample R2 was 0.4256. Our new in-sample R2 is 0.4352.
# This is not surprising:
# The linear regression model could "just" use the results of the collaborative filtering and ignore all the other independent variables.
# Since the linear regression model has access to so many other independent variables, it can do strictly better ON THE TRAINING SET
# What about the test set? Let us find out...

pred.lm.full <- predict(mod.lm.full, newdata=full.test)
R2.lm.full <- 1 - sum((pred.lm.full-full.test$rating)^2)/sum((mean(full.train$rating) - full.test$rating)^2)
R2.lm.full
R2.outsample

# The ensemble model does better than "just" the collaborative filtering model ON THE TEST SET
# The values reported here are slightly worse than those from the lecture, because we only used a subset of the independent variables.



